In [13]:
import h2o
from h2o.automl import H2OAutoML

## Visual Python Upgrade
NOTE: 
- Refresh your web browser to start a new version.
- Save VP Note before refreshing the page.

In [14]:
# Visual Python
!pip install visualpython --upgrade

In [15]:
# Visual Python
!visualpy install

Package install command: pip
Already exists Visual Python.
Disabling notebook extension visualpython/visualpython...
      - Validating: OK
Remove Visual Python Directories.
Target Dir :  /Users/sarayabesi/opt/anaconda3/share/jupyter/nbextensions
Copy visualpyhthon extension files ...
--------------------------------------------------------------------------------------------
Source Dir :  /Users/sarayabesi/opt/anaconda3/lib/python3.9/site-packages/visualpython
Target Dir :  /Users/sarayabesi/opt/anaconda3/share/jupyter/nbextensions/visualpython
Enabling notebook extension visualpython/visualpython...
      - Validating: OK


In [5]:
# Start the H2O cluster (locally)
#h2o.init()

# leave 1/3 for the RAM to XGBoost
h2o.init(max_mem_size = "10G")


#train = h2o.import_file(path="/Users/sarayabesi/Desktop/final_fixations_train.csv")
#test = h2o.import_file(path="/Users/sarayabesi/Desktop/final_fixations_test.csv")
data = h2o.import_file(path="/Users/sarayabesi/Documents/research-poly/paper_2/model_1/data_model_1.csv")

# Split the data into Train/Test/Validation with Train having 70% and test and validation 15% each
#train,test,valid = data.split_frame(ratios=[.7, .15])
train,test = data.split_frame(ratios=[.8])


Checking whether there is an H2O instance running at http://localhost:54321. connected.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,22 days 20 hours 38 mins
H2O_cluster_timezone:,America/Toronto
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.3
H2O_cluster_version_age:,9 months
H2O_cluster_name:,H2O_from_python_sarayabesi_dbu1zu
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,8.59 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [16]:
'number of rows in the whole dataset:' , len(data)

('number of rows in the whole dataset:', 222)

In [17]:
'number of rows in train:' , len(train)

('number of rows in train:', 175)

In [18]:
'number of rows in test:' , len(test)

('number of rows in test:', 47)

In [10]:
data.head(10)

participant_id,task_number,bug_number,fixation_rate_AOI_A1,fixation_rate_AOI_A2,fixation_rate_AOI_A3,fixation_rate_AOI_A4,fixation_rate_AOI_A5,fixation_rate_AOI_A6,avg_fixation_time_AOI_A1,avg_fixation_time_AOI_A2,avg_fixation_time_AOI_A3,avg_fixation_time_AOI_A4,avg_fixation_time_AOI_A5,avg_fixation_time_AOI_A6,is_accepted,is_correct
101,1,9,0.849711,0.115607,0.0693642,0,0,0.0346821,628.667,567.8,412.417,0,0,243.167,1,0
101,2,2,0.572438,0.427562,0.314488,0,0,0,659.043,887.116,869.461,0,0,0,0,1
101,3,8,0.92437,0.0756303,0.0504202,0,0,0,916.227,858.333,1200.67,0,0,0,1,1
101,4,15,0.822222,0.177778,0.162963,0,0,0,878.721,546.667,576.136,0,0,0,0,1
101,5,18,0.816,0.184,0.152,0,0,0,656.696,850.913,970,0,0,0,1,1
101,6,23,0.89899,0.0909091,0.0909091,0,0,0.010101,683.213,350.333,350.333,0,0,300,1,0
102,1,11,0.26484,0.182648,0.0182648,0.531202,0.022831,0.021309,608.695,782.217,354.25,707.163,287.667,458.429,1,0
102,2,15,0.229575,0.00571895,0,0,0,0.764706,624.249,267.429,0,0,0,469.998,1,1
102,3,4,0.321244,0,0,0,0,0.678756,382.419,0,0,0,0,338.198,1,1
102,4,23,0.141509,0,0,0.858491,0.313679,0,356.667,0,0,631.618,708.436,0,0,0


In [19]:
# Identify predictors and response
x = data.columns[3:] # exclude 'participant_id' , 'task_number' , 'bug_number'
y = "is_accepted"
x.remove(y)

In [21]:
# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [22]:
# Run AutoML for 20 base models
aml = H2OAutoML(max_models=4, seed=1)
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |
12:00:22.357: AutoML: XGBoost is not available; skipping it.
12:00:22.486: _min_rows param, The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 175.0.

███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGeneralizedLinearEstimator : Generalized Linear Modeling
Model Key: GLM_1_AutoML_3_20240523_120022


GLM Model: summary
    family    link    regularization              lambda_search                                                                 number_of_predictors_total    number_of_active_predictors    number_of_iterations    training_frame
--  --------  ------  --------------------------  ----------------------------------------------------------------------------  ----------------------------  -----------------------------  ----------------------  -----------------------------------------------
    binomial  logit   Ridge ( lambda = 0.03611 )  nlambda = 30, lambda.max = 28.455, lambda.min = 0.03611, lambda.1se = 0.1286  13                            13                             44                      AutoML_3_20240523_120022_training_py_7_sid_b6d5

ModelMetricsBinomialGLM: glm
** Reported on train data. **

MSE: 0.1461589902472496
RMSE: 0.38230745512904873
LogLoss: 0.45652715972236196
AUC: 0.8593464052287583
AUCPR: 0.8467612635777176
Gini: 0.7186928104575165
Null degrees of freedom: 174
Residual degrees of freedom: 161
Null deviance: 242.4586366104281
Residual deviance: 159.7845059028267
AIC: 187.7845059028267

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3541036070857532
       0    1    Error    Rate
-----  ---  ---  -------  ------------
0      58   27   0.3176   (27.0/85.0)
1      7    83   0.0778   (7.0/90.0)
Total  65   110  0.1943   (34.0/175.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.354104     0.83      109
max f2                       0.287145     0.894737  114
max f0point5                 0.653583     0.818386  88
max accuracy                 0.653583     0.811429  88
max precision                0.998073     1         0
max recall                   0.160617     1         152
max specificity              0.998073     1         0
max absolute_mcc             0.581249     0.62798   102
max min_per_class_accuracy   0.653583     0.811111  88
max mean_per_class_accuracy  0.653583     0.811438  88
max tns                      0.998073     85        0
max fns                      0.998073     89        0
max fps                      0.00747556   85        174
max tps                      0.160617     90        152
max tnr                      0.998073     1         0
max fnr                      0.998073     0.988889  0
max fpr                      0.00747556   1         174
max tpr                      0.160617     1         152

Gains/Lift Table: Avg response rate: 51.43 %, avg score: 51.43 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0114286                   0.94776            1.94444   1.94444            1                0.980155   1                           0.980155            0.0222222       0.0222222                  94.4444   94.4444            0.0222222
2        0.0228571                   0.929476           1.94444   1.94444            1                0.936994   1                           0.958574            0.0222222       0.0444444                  94.4444   94.4444            0.0444444
3        0.0342857                   0.867072           1.94444   1.94444            1                0.901786   1                           0.939645            0.0222222       0.0666667                  94.4444   94.4444            0.0666667


In [23]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows 

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
GLM_1_AutoML_3_20240523_120022,0.775556,0.523285,0.739793,0.214706,0.416441,0.173423
GBM_2_AutoML_3_20240523_120022,0.774248,0.560539,0.728686,0.238889,0.43294,0.187437
StackedEnsemble_AllModels_1_AutoML_3_20240523_120022,0.770588,0.533914,0.737668,0.214379,0.422513,0.178517
GBM_3_AutoML_3_20240523_120022,0.769412,0.55963,0.728034,0.238889,0.436222,0.19029
DRF_1_AutoML_3_20240523_120022,0.767647,0.570629,0.733379,0.244771,0.437696,0.191578
StackedEnsemble_BestOfFamily_1_AutoML_3_20240523_120022,0.763529,0.535406,0.729409,0.214379,0.423168,0.179071


In [24]:
# best model
#aml.leader

In [25]:
# prediction on the leader model
preds = aml.leader.predict(test)

glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [26]:
preds

predict,p0,p1
1,0.254051,0.745949
1,0.302242,0.697758
0,0.914888,0.085112
1,0.21438,0.78562
1,0.506843,0.493157
1,0.230252,0.769748
1,0.308262,0.691738
1,0.262551,0.737449
1,0.239852,0.760148
1,0.212226,0.787774


In [27]:
perf = aml.leader.model_performance(test)
perf

ModelMetricsBinomialGLM: glm
** Reported on test data. **

MSE: 0.21587323616492318
RMSE: 0.46462160535743835
LogLoss: 0.638546816096779
AUC: 0.650375939849624
AUCPR: 0.6717593844646195
Gini: 0.3007518796992481
Null degrees of freedom: 46
Residual degrees of freedom: 33
Null deviance: 64.67979226368345
Residual deviance: 60.02340071309723
AIC: 88.02340071309723

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5434236257350723
       0    1    Error    Rate
-----  ---  ---  -------  -----------
0      10   9    0.4737   (9.0/19.0)
1      3    25   0.1071   (3.0/28.0)
Total  13   34   0.2553   (12.0/47.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.543424     0.806452  33
max f2                       0.071103     0.886076  45
max f0point5                 0.543424     0.762195  33
max accuracy                 0.543424     0.744681  33
max precision                0.778968     0.833333  5
max recall                   0.071103     1         45
max specificity              0.813616     0.947368  0
max absolute_mcc             0.543424     0.459873  33
max min_per_class_accuracy   0.681594     0.578947  24
max mean_per_class_accuracy  0.543424     0.709586  33
max tns                      0.813616     18        0
max fns                      0.813616     28        0
max fps                      0.0024276    19        46
max tps                      0.071103     28        45
max tnr                      0.813616     0.947368  0
max fnr                      0.813616     1         0
max fpr                      0.0024276    1         46
max tpr                      0.071103     1         45

Gains/Lift Table: Avg response rate: 59.57 %, avg score: 57.25 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0212766                   0.805662           0         0                  0                0.813616   0                           0.813616            0               0                          -100      -100               -0.0526316
2        0.0212766                   0.797708           0         0                  0                0          0                           0.813616            0               0                          -100      -100               -0.0526316
3        0.0425532                   0.796167           1.67857   0.839286           1                0.796325   0.5                         0.804971            0.0357143       0.0357143                  67.8571   -16.0714           -0.0169173
4        0.0425532                   0.795975           0         0.839286           0                0          0.5                         0.804971            0               0.0357143                  -100      -16.0714           -0.0169173
5        0.0638298                   0.793468           1.67857   1.11905            1                0.795908   0.666667                    0.80195             0.0357143       0.0714286                  67.8571   11.9048            0.018797
6        0.106383                    0.781629           1.67857   1.34286            1                0.786697   0.8                         0.795849            0.0714286       0.142857                   67.8571   34.2857            0.0902256
7        0.148936                    0.770435           0.839286  1.19898            0.5              0.777792   0.714286                    0.79069             0.0357143       0.178571 

In [28]:
# Get leaderboard with all possible columns
# log loss calculation is for validation data
lb = h2o.automl.get_leaderboard(aml, extra_columns = "ALL")
lb

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse,training_time_ms,predict_time_per_row_ms,algo
GLM_1_AutoML_3_20240523_120022,0.775556,0.523285,0.739793,0.214706,0.416441,0.173423,14,0.045425,GLM
GBM_2_AutoML_3_20240523_120022,0.774248,0.560539,0.728686,0.238889,0.43294,0.187437,47,0.033771,GBM
StackedEnsemble_AllModels_1_AutoML_3_20240523_120022,0.770588,0.533914,0.737668,0.214379,0.422513,0.178517,429,0.060459,StackedEnsemble
GBM_3_AutoML_3_20240523_120022,0.769412,0.55963,0.728034,0.238889,0.436222,0.19029,39,0.021,GBM
DRF_1_AutoML_3_20240523_120022,0.767647,0.570629,0.733379,0.244771,0.437696,0.191578,65,0.020497,DRF
StackedEnsemble_BestOfFamily_1_AutoML_3_20240523_120022,0.763529,0.535406,0.729409,0.214379,0.423168,0.179071,429,0.031464,StackedEnsemble
